In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=ae0cd0a1cde873c613cf8126730296b0fcbc2059e9c882900d3ad972a128e15b
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('lrex').getOrCreate()
df=spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)

In [ ]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [ ]:
for ship in df.head(5):
  print(ship)
  print('\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)




In [ ]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors

In [ ]:
indexer=StringIndexer(inputCol='Cruise_line',outputCol='cruise_category')
indexed=indexer.fit(df).transform(df)

In [ ]:
indexed.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_category=16.0)]

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_category']

In [ ]:
assembler=VectorAssembler(inputCols=['Age','Tonnage',
                                     'passengers','length',
                                     'cabins','passenger_density','cruise_category'],
                          outputCol='features')

output=assembler.transform(indexed)

In [ ]:
final_data=output.select('features','crew')

In [ ]:
(train_data,test_data)=final_data.randomSplit([0.7,0.3])

In [ ]:
train_data.describe().show()


+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               115|
|   mean| 7.957826086956533|
| stddev|3.6290759982491263|
|    min|               0.6|
|    max|              21.0|
+-------+------------------+



In [ ]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                43|
|   mean| 7.356511627906976|
| stddev|3.1409930820567302|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
ship_lr=LinearRegression(labelCol='crew')

In [ ]:
trained_ship_model=ship_lr.fit(train_data)

In [ ]:
ship_results=trained_ship_model.evaluate(test_data)

In [ ]:
ship_results.rootMeanSquaredError

0.68715491180134

In [ ]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                43|
|   mean| 7.356511627906976|
| stddev|3.1409930820567302|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



In [ ]:
ship_results.r2

0.9510001764856769

In [ ]:
ship_results.meanSquaredError

0.47218187281270746

In [ ]:
ship_results.meanAbsoluteError

0.5314976138264943

In [ ]:
from pyspark.sql.functions import corr

In [ ]:
df.select(corr('cabins','crew')).show()

+------------------+
|corr(cabins, crew)|
+------------------+
|0.9508226063578497|
+------------------+



In [ ]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [ ]:
df.select(corr('passengers','crew')).show()

+----------------------+
|corr(passengers, crew)|
+----------------------+
|    0.9152341306065384|
+----------------------+

